<img src="https://drive.google.com/uc?id=181S0KcAdGeAajZ1apcbOvoh3wvYuKtzd">

<a href="https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/01.00-IPython-Beyond-Normal-Python.ipynb"><img align="left"  src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>&nbsp; &nbsp;<a href=""><img align="left" src="http://www.lapix.ufsc.br/wp-content/uploads/2019/04/License-CC-BY-ND-4.0-orange.png" alt="Creative Commons 4.0 License" title="Creative Commons 4.0 License"></a>&nbsp; &nbsp; <a href=""><img align="left" src="http://www.lapix.ufsc.br/wp-content/uploads/2019/04/Jupyter-Notebook-v.1.0-blue.png" alt="Jupyter Version" title="Jupyter Version"></a>&nbsp; &nbsp;<a href=""><img align="left"  src="https://img.shields.io/badge/python-3.10-green" alt="Python Version" title="Python Version"></a>

## Autores:

*   Aldo von Wangenheim (aldo.vw@ufsc.br)
*   Rodrigo de Paula e Silva Ribeiro (ribeiro.rodrigo@posgrad.ufsc.br)


CERTIFIQUE-SE DE TER INCIADO O AMBIENTE EM MODO <b>GPU</b> ANTES DE REALIZAR OS PROCEDIMENTOS.<br>
NA TROCA DO AMBIENTE ELE <b><u>PERDE</u></b> TODOS OS DADOS BAIXADOS.

AO RE-EXECUTAR OS PROCEDIMENTOS (ex. diminuir o batch size), <br>REINICIE O AMBIENTE PARA LIBERAR O CACHE DO PYTORCH (reiniciar não perde os dados da sessão).<br>(alt+m ou pelo menu "Ambiente de Execução")

EXECUTE UTILIZANDO RUN ALL ou CTRL+F9

In [ ]:
!nvidia-smi
!nvcc --version

### Selecionar GPU e PATH

In [ ]:
!nvcc --version

try:
    import google.colab
    _ON_COLAB = True
except:
    _ON_COLAB = False
    %reload_ext autoreload
    %autoreload 2
    %matplotlib inline
    from IPython import get_ipython
    
print('Running on Google Colab = ', _ON_COLAB)
import os
import torch

# Definir o caminho base
if _ON_COLAB:
    BASE_PATH = "/content"
    import locale
    locale.getpreferredencoding = lambda: "UTF-8"
else:
    BASE_PATH = "/lapix" # trocar para o base path do seu computador se estiver rodando localmente, deixar /lapix se estiver rodando nos conteiners lapix
    VISIBLE_GPUS = [7] # Selecionar a GPU PARA RODAR! VER QUAL ESTA LIVRE

    if torch.cuda.device_count() != 8:
        print("GPU SETADA - PULANDO ETAPA")
    else:
        os.environ["CUDA_VISIBLE_DEVICES"] = str(VISIBLE_GPUS).replace("]", "").replace("[","").replace(" ", "")
        os.environ["NVIDIA_VISIBLE_DEVICES"] = str(VISIBLE_GPUS).replace("]", "").replace("[","").replace(" ", "")
        print("CUDA GPUS NUMBER: ", torch.cuda.device_count())
    
os.chdir(BASE_PATH) # garantir que está executando no caminho base definido

# O que é YOLO?
YOLO (“You Only Look Once”), é uma série de detectores de objetos de processamento rápido que vem sendo desenvolvida por diversos autores. Ao contrário dos detectores tradicionais que utilizam áreas de interesse para propor regiões (ROI) que podem conter um objeto na imagem (conhecidos como detectores de 2-estágios), a arquitetura YOLO divide a imagem em grids e vasculha a imagem como um todo (detector de um único estágio).

Neste notebook didático, é utilizado a versão 7 desta arquitetura.<br>
Algumas notas sobre a versão 7:<br>
* Em versões prévias era apenas possível utilizar detecção baseada em bounding box, já na versão 7 é possível utilizar também segmentação por instância e keypoint.
* Na versão 7, utiliza-se apenas o método OneCycle para otimização da Learning Rate, portanto não é necessário definir uma Learning Rate, a não ser que seja desejável utilizar Learning Rate fixa (linear).
* Foi conceitualizada pelos mesmos autores da YoloV4, Scaled-YoloV4 e YoloR. De fato é uma versão baseada na Scaled-YoloV4 com muitas melhorias.
* Diferente de outras versões que os pesos eram pré-treinados no conjunto imagenet, a versão 7, assim como a Scaled YoloV4, é pré-treinada no [COCO Dataset](https://cocodataset.org/#home).

Diferentemente do Detectron2 (framework customizável), YOLO é uma arquitetura parametrizada e única, embora é possível alterar diversos parâmetros no arquivo train.py oficial, muitas alterações ou valores muito diferentes podem descaracterizar a rede perdendo a sua capacidade de atingir resultados SOTA (state of the art).

Neste notebook apenas alguns parâmetros de performance são editáveis, para editar parâmetros mais específicos ou até mesmo implementar modificações, é recomendável ler e se familiarizar com a arquitetura.

Links:<br>
[Link Artigo](https://arxiv.org/abs/2207.02696) <br>
[Link git](https://github.com/WongKinYiu/yolov7)


# Instalação do YoloV7 do repositório oficial e suas dependências.

In [ ]:
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt

# Montar o google drive
As 'runs' de treino serão salvas na pasta "aula_yolov7" no seu drive.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

# Baixar o dataset de cones já no formato YOLO
O dataset de cones tem por objetivo possibilitar identificar diferentes tipos de cones sinalizadores de trânsito.<br>
É um conjunto anotado no formato "bounding box" para detecção de objetos.<br>
O conjunto possui 5 classes distintas:
* blue_cone
* large_orange_cone
* orange_cone
* yellow_cone
* unknown_cone

Amostra de anotações:<br>
<img src="https://drive.google.com/uc?id=1WgjF0WaRLahBQyLRCAWScPSpIAFkgzJu">
<img src="https://drive.google.com/uc?id=1WgAlliwetRb4zBE_0AUMQvxrtjp4A4_e">
<img src="https://drive.google.com/uc?id=15t9g5d9PfRJx_Iss7qhP1vvBHPbNgQoX">
<img src="https://drive.google.com/uc?id=1KErMuro6T0_w17d6Gqfgtwv76h6CarNW">

In [ ]:
import os
os.chdir("/content/")
!gdown 11bEzhQELv4bFUBaQEN_fPdc67BPnKwcq
os.makedirs("/content/dataset", exist_ok=True)
%mv yolo_v7_cones_dataset.zip /content/dataset/
os.chdir("/content/dataset")
!unzip -qq -u yolo_v7_cones_dataset.zip
!rm -rf yolo_v7_cones_dataset.zip
os.chdir("/content/")

# Baixar o peso do modelo padrão para transfer learning.
Para mais opções de pesos pré-treinados e modelos maiores/menores, ver o github oficial do [YoloV7](https://github.com/WongKinYiu/yolov7).

In [ ]:
%cd /content/yolov7
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

# Parâmetros básicos
Implementação de edição dos parâmetros básicos de performance do Yolo v7.<br>
Para parâmetros opcionais e outros, ver [este código do git (linhas 527 à 565)](https://github.com/WongKinYiu/yolov7/blob/main/train.py) e adequar o código.<br>



In [ ]:
# Definir onde salvar os resultados de treino
SAVE_PATH = "/content/gdrive/MyDrive/aula_yolov7/"
os.makedirs(SAVE_PATH, exist_ok=True)

# Definir o batch size
BATCH_SIZE = 16

# Definir o número de épocas de treino
EPOCHS = 1

# Definir o formato de imagem para rede
IM_SIZE = "640 640" # padrão 640 640, imagem menor = mais rápido, menos preciso em objetos pequenos; imagem maior = mais lento, mais preciso em objetos pequenos.

In [ ]:
%cd /content/yolov7
!python train.py --batch $BATCH_SIZE --epochs $EPOCHS --img-size $IM_SIZE --data "/content/dataset/data.yaml" --weights 'yolov7_training.pt' --device 0 --project $SAVE_PATH

# Realizar inferência de métricas no conjunto de validação (val/valid)

In [ ]:
curr_weight = SAVE_PATH + "exp/weights/best.pt" # trocar exp por exp1/exp2/exp3 conforme o número do último treino gerado. É incrementado automaticamente.
!python test.py --data "/content/dataset/data.yaml" --batch $BATCH_SIZE --conf-thres 0.5 --iou-thres 0.5 --device 0 --weights "$curr_weight" --name yolov7_validation

# Visualização dos gráficos das métricas

In [ ]:
from glob import glob
metrics_img = glob("/content/yolov7/runs/test/yolov7_validation/*.png")
for mi in metrics_img:
  img_m = cv2.imread(mi)
  cv2_imshow(img_m)


# Visualizar detecções no conjunto de teste (test)

In [ ]:
curr_weight = SAVE_PATH + "exp/weights/best.pt" # trocar exp por exp1/exp2/exp3 conforme o número do último treino gerado. É incrementado automaticamente.
!python detect.py --weights "$curr_weight" --conf 0.5 --source "/content/dataset/test/images"

In [ ]:
from glob import glob
import os, cv2, random
from google.colab.patches import cv2_imshow
num_samples = 4 # numero de imagens para visualizar.
all_dets = glob('/content/yolov7/runs/detect/exp/*.jpg')
for det in random.sample(all_dets, num_samples):
  det_img = cv2.imread(det)
  cv2_imshow(det_img)

<img src="http://lapix.ufsc.br/wp-content/uploads/2022/10/rodape-lapix.png">